In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
run_200_1000 = pd.read_csv('data/20211026T184846_200-1000-iperf.csv')

In [7]:
def pckt_count(a):
    a = a.split(';')
    a = [int(i) for i in a[:-1]]
    return len(a)

run_200_1000['pck_ct'] = run_200_1000['packet_times'].apply(pckt_count)

run_200_1000['packet_size_total'] = run_200_1000['1->2Pkts'] + run_200_1000['2->1Pkts']

ratio_200_1000 = [0.2 for i in range(318)]

run_200_1000['ratio'] = ratio_200_1000

def agg_10sec(df):
    new_df = pd.DataFrame()
    min_time = df["Time"][0]
    ratio = df["ratio"][0]
    while min_time < df["Time"][len(df)-1]:
        temp_df = df[(df["Time"] >= min_time) & (df["Time"] < min_time+10)]
        row = temp_df[["1->2Bytes", "2->1Bytes", "1->2Pkts", "2->1Pkts", "packet_size_total", "pck_ct"]].sum().to_frame().T
        row["packet_sizes_var"] = temp_df["packet_size_total"].var()
        row["Time"] = min_time
        row["ratio"] = ratio
        new_df = new_df.append(row)
        min_time += 10
    return new_df.reset_index(drop=True)

agg_200_1000 = agg_10sec(run_200_1000)

agg_200_1000['latency'] = [200] * len(agg_200_1000)

In [6]:
agg_200_1000['latency'] = [200] * len(agg_200_1000)
agg_200_1000

,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_size_total,pck_ct,packet_sizes_var,Time,ratio,latency
0,11679296,127752,7790,2312,10102,10102,711988.515152,1635274127,0.2,200
1,4971056,37892,3315,713,4028,4028,23062.763636,1635274137,0.2,200
2,3742556,28660,2496,541,3037,3037,13355.090909,1635274147,0.2,200
3,3777000,30764,2518,584,3102,3102,2652.177778,1635274157,0.2,200
4,3621000,36716,2414,695,3109,3109,2734.988889,1635274167,0.2,200
5,3802556,36868,2536,700,3236,3236,10600.363636,1635274177,0.2,200
6,3667500,32296,2445,607,3052,3052,1159.511111,1635274187,0.2,200
7,3786000,50600,2524,962,3486,3486,2038.933333,1635274197,0.2,200
8,3679500,44268,2453,834,3287,3287,1827.122222,1635274207,0.2,200
9,3777000,43264,2518,823,3341,3341,1325.655556,1635274217,0.2,200


In [ ]:
###ENTIRE DATASET
full_df = pd.concat([agg_200_300, agg_200_1000, agg_200_5000, agg_200_25000])

# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var', 'Time']
df_X = full_df[features]
df_y = full_df['ratio']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))